<a href="https://colab.research.google.com/github/Kulabus/Kulabus/blob/main/Knowledge_Rag_%D0%B1%D0%BE%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>RAG-бот на Knowledge Graph с интерфейсом Gradio

На основе модели saiga_mistral_7b из портала HuggingFace соберём модель чат-бота, отвечающего на вопросы по теме выбранной статьи из Википедии. Ридер загружает данные из страницы Википедии в графовую базу данных. В предобученную модель Saiga поступает вопрос в шаблоне (промпт). Модель, используя полученную графовую базу и собственные "знания", отвечает на вопрос.

In [ ]:
%%writefile requirements.txt
transformers>=4.42.0
llama_index==0.10.46
pyvis==0.3.2
Ipython==7.34.0
langchain==0.2.5
pypdf==4.2.0
langchain_community==0.2.5
llama-index-llms-huggingface==0.2.3
llama-index-embeddings-huggingface==0.2.2
llama-index-embeddings-langchain==0.1.2
langchain-huggingface==0.0.3
sentencepiece==0.1.99
accelerate==0.31.0
bitsandbytes==0.43.1
peft==0.11.1
llama-index-readers-wikipedia==0.1.4
wikipedia==1.4.0
llama-index-readers-web==0.1.9
gradio==4.44.1

# Зависимости
huggingface-hub==0.23.3
torch==2.3.1
numpy==1.25.2
packaging==24.1
pyyaml==6.0.1
requests==2.31.0
tqdm==4.66.4
filelock==3.14.0
regex==2024.5.15
typing-extensions==4.12.2
safetensors==0.4.3
tokenizers==0.19.1


Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.15.1
    Uninstalling gradio_client-0.15.1:
      Successfully uninstalled gradio_client-0.15.1
  Attempting uninstall: gradio
    Found existing installation: gradio 4.27.0
    Uninstalling gradio-4.27.0:
      Successfully uninstalled gradio-4.27.0


In [ ]:
import gradio as gr
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.embeddings.langchain import LangchainEmbedding
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch
from pyvis.network import Network
from huggingface_hub import login
from langchain_huggingface import HuggingFaceEmbeddings

# Инициализация модели (вынесено в отдельную функцию для удобства)
def init_model():
    HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN"
    login(HF_TOKEN, add_to_git_credential=True)

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
    config = PeftConfig.from_pretrained(MODEL_NAME)

    model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        quantization_config=quantization_config,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float16)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

    llm = HuggingFaceLLM(
        model=model,
        model_name=MODEL_NAME,
        tokenizer=tokenizer,
        max_new_tokens=generation_config.max_new_tokens,
        model_kwargs={"quantization_config": quantization_config},
        generate_kwargs = {
            "bos_token_id": generation_config.bos_token_id,
            "eos_token_id": generation_config.eos_token_id,
            "pad_token_id": generation_config.pad_token_id,
            "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
            "repetition_penalty": generation_config.repetition_penalty,
            "temperature": generation_config.temperature,
            "do_sample": True,
            "top_k": 50,
            "top_p": 0.95
        },
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="auto",
    )

    embed_model = LangchainEmbedding(
        HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    return llm, embed_model

# Функции для промптов
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

# Функция для обработки запроса
def process_query(wiki_page, query_text):
    # Инициализация модели (если еще не инициализирована)
    if not hasattr(process_query, 'initialized'):
        init_model()
        process_query.initialized = True

    # Загрузка данных из википедии
    reader = WikipediaReader()
    docs = reader.load_data(pages=[wiki_page], lang_prefix='ru')

    # Создание графового хранилища и индекса
    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    indexKG = KnowledgeGraphIndex.from_documents(
        documents=docs,
        max_triplets_per_chunk=3,
        show_progress=True,
        include_embeddings=True,
        storage_context=storage_context
    )

    # Создание шаблона сообщения
    message_template = f"""<s>system
Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если нет, то просто скажи: 'я не знаю'. Не придумывай! </s>
<s>user
Вопрос: {query_text}
Источник:
</s>
"""

    # Выполнение запроса
    query_engine = indexKG.as_query_engine(include_text=True, verbose=True)
    response = query_engine.query(message_template)

    return response.response


In [ ]:
# Создание интерфейса Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Генерация ответов на основе статей из Википедии")

    with gr.Row():
        wiki_input = gr.Textbox(label="Статья из Википедии", value="Аристотель")
        query_input = gr.Textbox(label="Ваш запрос", value="Придумай два четверостишья с перекрестной рифмой на тему философии Аристотеля")

    submit_btn = gr.Button("Отправить запрос")
    output = gr.Textbox(label="Ответ модели", interactive=False)

    submit_btn.click(
        fn=process_query,
        inputs=[wiki_input, query_input],
        outputs=output
    )

# Запуск интерфейса
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2b9bd02e90a538a999.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

...

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Extracted keywords: ['Аристотель', 'четверостишие', 'рифма', 'КЕЙВАРДС: философия', 'перекрестная', 'философия', 'перекрестная рифма', 'КЕЙВАРДС']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Аристотель', 'Сформулировал', 'Правила составления силлогических фигур']
['Аристотель', 'Положил основание', 'Первым логическим принципам']
['Аристотель', 'Отличался склонностьью спорить с', 'Учителем']
['Аристотель', 'Отличался склонностью спорить с', 'Учителем']
['Аристотель', 'Оставался в школе платона', 'До его смерти']
['Аристотель', 'Происходили разногласия между', 'Платоном']
['Аристотель', 'Обучался ораторскому искусству', 'Исократ']
['Аристотель', 'Происходили разногласии между', 'Платоном']
('Аристотель', 'Подходит к', 'Идее единичного бытия вещи')
['Аристотель', 'Впоследствии', 'Воплотился в сочинениях']
('Аристотель', 'Аналитика', 'Умение свести рассуж